In [1]:
import numpy as np
from torch.nn.functional import conv_transpose2d as tran
from torch.nn.functional import conv2d as libConv2d
import torch

In [2]:
in_data = torch.randn(1, 1, 2, 2)  
w = torch.randn(1, 1, 2, 2)  
out = tran(in_data, w, stride=1, padding=0, dilation=1)
stride = 1
padding = 0
dilattion = 1

In [3]:
import numpy as np

def conv2d_transposed(in_data, in_channels, out_channels, kernel_size,
                      stride=1, padding=0, output_padding=0, dilation=1,
                      bias=True, padding_mode='zeros'):
    # Проверка и создание bias
    if bias:
        bias_val = np.random.rand(out_channels)
    else:
        bias_val = np.zeros(out_channels)
    # Проверка padding_mode
    if padding_mode != 'zeros':
        raise ValueError('only "zeros" padding_mode in ConvTranspose2d')
    # Генерация ядра
    if type(kernel_size) == tuple:
        weights = np.random.rand(out_channels, in_channels, kernel_size[0], kernel_size[1])
    elif type(kernel_size) == int:
        weights = np.random.rand(out_channels, in_channels, kernel_size, kernel_size)
    else:
        raise ValueError('Invalid kernel_size type')
    # Транспонированная свертка вручную
    result_tensor = []
    for out_channel_idx in range(out_channels):
        feature_map = np.zeros(((in_data.shape[2] - 1) * stride + dilation * (kernel_size - 1) + 1,
                                (in_data.shape[3] - 1) * stride + dilation * (kernel_size - 1) + 1))
        for in_channel_idx in range(in_channels):
            for i in range(0, in_data.shape[2]):
                for j in range(0, in_data.shape[3]):
                    value = in_data[0, in_channel_idx, i, j]
                    product = value * weights[out_channel_idx, in_channel_idx]
                    zero_tensor = np.zeros(((weights.shape[2] - 1) * dilation + 1, (weights.shape[3] - 1) * dilation + 1))

                    for a in range(0, zero_tensor.shape[0], dilation):
                        for b in range(0, zero_tensor.shape[1], dilation):
                            zero_tensor[a, b] = product[a // dilation, b // dilation]

                    res = np.add(zero_tensor, feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                                                   j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1])
                    feature_map[i * stride:i * stride + (weights.shape[2] - 1) * dilation + 1,
                    j * stride:j * stride + (weights.shape[3] - 1) * dilation + 1] = res
        result_tensor.append(np.add(feature_map, bias_val[out_channel_idx]))
    for t in range(len(result_tensor)):
        if output_padding > 0:
            result_tensor[t] = np.pad(result_tensor[t], ((0, output_padding), (0, output_padding)), 'constant', constant_values=0)
        result_tensor[t] = result_tensor[t][padding:-padding, padding:-padding]
    return result_tensor, weights, bias_val


In [4]:
import torch
import numpy as np

def compare(tensor, in_channels, out_channels, kernel_size, stride, padding, output_padding, dilation, bias=True, padding_mode='zeros'):
    my_res, my_kernel, my_bias_val = conv2d_transposed(
        tensor,
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
    )

    torch_function = torch.nn.ConvTranspose2d(
        in_channels=in_channels, out_channels=out_channels,
        kernel_size=kernel_size, stride=stride,
        padding=padding, output_padding=output_padding,
        dilation=dilation, bias=bias,
        padding_mode=padding_mode,
    )
    torch_function.weight.data = torch.Tensor(my_kernel)
    torch_function.bias.data = torch.Tensor(my_bias_val)

    my_result = str(np.round([tensor.tolist() for tensor in my_res], 2))
    torch_result = str(np.round(torch_function(torch.Tensor(tensor)).data.numpy(), 2))
    return my_result == torch_result



In [5]:
tensor = np.random.randn(1, 1, 4, 4)
in_channels = 1
out_channels = 1
kernel_size = 2
stride = 1
padding = 0
output_padding = 0
dilation = 1
result = compare(tensor, in_channels, out_channels, kernel_size, stride, padding, output_padding, dilation)
print(result)

True


In [6]:
tensor = np.random.randn(1, 1, 5, 5)
in_channels = 1
out_channels = 1
kernel_size = 3
stride = 2
padding = 1
output_padding = 1
dilation = 1
result = compare(tensor, in_channels, out_channels, kernel_size, stride, padding, output_padding, dilation)
print(result)

True


+++

In [7]:
import numpy as np
import torch

def custom_transpose_conv2d(
    input_tensor, in_channels, out_channels, kernel_size,
    transp_stride=1, padding=0, dilation=1, use_bias=True, padding_mode='zeros'
):
    pad = (kernel_size - 1) * dilation - padding

    input_tensor_padded = np.pad(input_tensor, ((0, 0), (pad, pad), (pad, pad)), mode=padding_mode)

    
    if use_bias:
        bias_val = np.random.rand(out_channels)
    else:
        bias_val = np.zeros(out_channels)

    weights = np.random.rand(out_channels, in_channels, kernel_size, kernel_size)

    output_size = (
        (input_tensor.shape[1] - 1) * transp_stride + dilation * (kernel_size - 1) + 1,
        (input_tensor.shape[2] - 1) * transp_stride + dilation * (kernel_size - 1) + 1,
    )

    result_tensor = np.zeros((out_channels,) + output_size)

    for l in range(out_channels):
        for c in range(in_channels):
            for i in range(0, input_tensor.shape[1]):
                for j in range(0, input_tensor.shape[2]):
                    val = input_tensor[c, i, j]
                    proizv = val * weights[l, c]
                    zero_tensor = np.zeros(
                        (weights.shape[2] - 1) * dilation + 1, (weights.shape[3] - 1) * dilation + 1
                    )
                    for a in range(0, zero_tensor.shape[0], dilation):
                        for b in range(0, zero_tensor.shape[1], dilation):
                            zero_tensor[a, b] = proizv[a // dilation, b // dilation]

                    result_tensor[l, i * transp_stride : i * transp_stride + zero_tensor.shape[0], 
                                 j * transp_stride : j * transp_stride + zero_tensor.shape[1]] += zero_tensor

        result_tensor[l] += bias_val[l]

    return result_tensor, weights, bias_val


In [8]:
import torch

def compare_with_torch(input_tensor, in_channels, out_channels, kernel_size, transp_stride=1, padding=0, dilation=1, use_bias=True, padding_mode='zeros'):
    result_custom, weights_custom, bias_custom = custom_transpose_conv2d(
        input_tensor, in_channels, out_channels, kernel_size,
        transp_stride, padding, dilation, use_bias, padding_mode
    )

    conv_transpose = torch.nn.ConvTranspose2d(
        in_channels, out_channels, kernel_size,
        stride=transp_stride, padding=padding, dilation=dilation, bias=use_bias, padding_mode=padding_mode
    )
    conv_transpose.weight.data = weights_custom
    conv_transpose.bias.data = bias_custom

    result_torch = conv_transpose(torch.tensor(input_tensor)).data.numpy()

    return np.allclose(result_custom, result_torch)



In [9]:
input_tensor1 = np.random.rand(1, 4, 4, 4)  
in_channels1 = 3
out_channels1 = 2
kernel_size1 = 3
result_comparison1 = compare_with_torch(input_tensor1, in_channels1, out_channels1, kernel_size1)
print(result_comparison1)



True


In [10]:
input_tensor2 = np.random.rand(1, 1, 2, 2)  
in_channels2 = 1
out_channels2 = 3
kernel_size2 = 2
result_comparison2 = compare_with_torch(input_tensor2, in_channels2, out_channels2, kernel_size2)
print(result_comparison2)

True


In [11]:
input_tensor3 = np.random.rand(1, 2, 3, 3) 
in_channels3 = 2
out_channels3 = 2
kernel_size3 = 2
result_comparison3 = compare_with_torch(input_tensor3, in_channels3, out_channels3, kernel_size3)
print(result_comparison3)

True
